In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.metrics import accuracy_score, confusion_matrix,roc_curve,roc_auc_score
from scipy.stats import zscore
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.linear_model import Ridge,Lasso,RidgeCV,LassoCV
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import PowerTransformer
from scipy.stats import skew

In [ ]:
df=pd.read_csv(r"https://raw.githubusercontent.com/dsrscientist/Data-Science-ML-Capstone-Projects/master/baseball.csv")

In [ ]:
df

In [ ]:
df.rename(columns={'W':'Predicted Wins','R':'Runs','AB':'At Bats','H':'Hits','2B':'Doubles','3B':'Triples','HR':'Homeruns','BB':'Walks','SO':'Strikeouts','SB':'Stolen Bases','RA':'Runs Allowed','ER':'Earned Runs','ERA':'Earned Run Average', 'CG':'Compelete Games','SHO':'Shutouts','SV':'Saves', 'E':'Error'},inplace=True)

In [ ]:
df

In [ ]:
df.columns

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
sns.pairplot(df)

In [ ]:
df.plot(kind='box',subplots=True,layout=(5,4),figsize=(25,25))

In [ ]:
X=df.drop(columns="Predicted Wins")
Y=df["Predicted Wins"]

In [ ]:
plt.figure(figsize=(20,30))
pltnumber=1

for column in X:
    if pltnumber<=15:
        ax=plt.subplot(5,3,pltnumber)
        plt.scatter(X[column],Y)
        plt.xlabel(column,fontsize=20)
        plt.ylabel('Predicted Wins',fontsize=20)
    pltnumber+=1
plt.tight_layout()

In [ ]:
for col in df:
    print(skew(df[col]))
    plt.figure()
    sns.distplot(df[col])
    plt.show

In [ ]:
df["Runs"]=np.sqrt(df["Runs"])
df.skew()

In [ ]:
df_corr=df.corr().abs()
df_corr
plt.figure(figsize=(25,30))
sns.heatmap(df_corr,annot=True)
plt.show()


In [ ]:
from sklearn.feature_selection import SelectPercentile
from sklearn.feature_selection import chi2

In [ ]:
Spercentile=SelectPercentile(score_func=chi2,percentile=80)
Spercentile=Spercentile.fit(X,Y)
cols=Spercentile.get_support(indices=True)
feature=X.columns[cols]
df_scores=pd.DataFrame({'Features':X.columns,'Chi2Score':Spercentile.scores_,'pvalues':Spercentile.pvalues_})
df_scores.sort_values(by='Chi2Score',ascending=False)
print(cols)
print(feature)

In [ ]:
df=df.drop(["At Bats","Doubles","Earned Run Average","Shutouts"],axis=1)

In [ ]:
df

# Model Training Using Linear Regression Without HyperParamter Tuning

In [ ]:
X=df.drop(columns="Predicted Wins")
Y=df["Predicted Wins"]

In [ ]:
scaler=StandardScaler()
X_scaled=scaler.fit_transform(X)

In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(X_scaled,Y,test_size=0.25,random_state=21)

In [ ]:
lm=LinearRegression()
lm.fit(X_train,Y_train)

In [ ]:
Y_pred=lm.predict(X_test)
Y_pred_train=lm.predict(X_train)

In [ ]:
lm.score(X_train,Y_train)
lm.score(X_test,Y_test)

In [ ]:
import statsmodels.formula.api as smf

In [ ]:
est=smf.ols(formula='Y~X',data=df).fit()
est.summary()

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
r2=r2_score(Y_test,Y_pred)

In [ ]:
r2

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
cross_val=cross_val_score(lm,X_train,Y_train,scoring="r2",cv=5)
cross_val

In [ ]:
cross_val.mean()

In [ ]:
mean_absolute_error(Y_test,Y_pred)


In [ ]:
mean_squared_error(Y_test,Y_pred)

In [ ]:
np.sqrt(mean_squared_error(Y_test,Y_pred))

In [ ]:
plt.scatter(Y_test,Y_pred)
plt.xlabel('actual data')
plt.ylabel('predicted data')
plt.show()

# Model Training Using Linear Regression With HyperParamter Tuning

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
ridge=Ridge()

In [ ]:
parameters={"alpha":[1e-15,1e-10,1e-8,1e-3,1e-2,1,5,10,20,30,35,40,45,50,55,100]}
ridge_regressor=GridSearchCV(ridge,parameters,scoring="r2",cv=5)

In [ ]:
ridge_regressor.fit(X_train,Y_train)

In [ ]:
print(ridge_regressor.best_params_)


In [ ]:
print(ridge_regressor.best_score_)

In [ ]:
prediction_ridge=ridge_regressor.predict(X_test)
prediction_ridge

In [ ]:
ridge_regressor.score(X_test,Y_test)

In [ ]:
y_pred=ridge_regressor.predict(X_test)

In [ ]:
r2=r2_score(Y_test,y_pred)

In [ ]:
r2

In [ ]:
cross_val=cross_val_score(ridge_regressor,X_train,Y_train,scoring="r2",cv=5)
cross_val

In [ ]:
cross_val.mean()

In [ ]:
mean_absolute_error(Y_test,y_pred)


In [ ]:
mean_squared_error(Y_test,y_pred)

In [ ]:
np.sqrt(mean_squared_error(Y_test,y_pred))

In [ ]:
plt.scatter(Y_test,y_pred)
plt.xlabel('actual data')
plt.ylabel('predicted data')
plt.show()

# Model Training Using KNN Without HyperParamter Tuning


In [ ]:
X=df.drop(columns="Predicted Wins")
Y=df["Predicted Wins"]

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
scaler=StandardScaler()

In [ ]:
X_scaled=scaler.fit_transform(X)

In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(X_scaled,Y,test_size=0.25,random_state=21)

In [ ]:
knn=KNeighborsRegressor()

In [ ]:
knn.fit(X_train,Y_train)

In [ ]:
Y_pred=knn.predict(X_test)

In [ ]:
knn.score(X_train,Y_train)

In [ ]:
knn.score(X_test,Y_test)

In [ ]:
r2=r2_score(Y_test,Y_pred)
r2

In [ ]:
plt.scatter(Y_test,Y_pred)
plt.xlabel('actual data')
plt.ylabel('predicted data')
plt.show()

# KNN MODEL USING HYPERPARAMETER TUNING

In [ ]:
X=df.drop(columns="Predicted Wins")
Y=df["Predicted Wins"]

In [ ]:
scaler=StandardScaler()

In [ ]:
X_scaled=scaler.fit_transform(X)

In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(X_scaled,Y,test_size=0.25,random_state=21)

In [ ]:
knn=KNeighborsRegressor()

In [ ]:
param_grid=[
    {'algorithm':['kd_tree','brute','ball_tree'],
    'leaf_size':[15,18,20],
    'n_neighbors':[3,5,7,13]
    }
]

In [ ]:
clf=GridSearchCV(knn,param_grid,cv=3,n_jobs=-1)

In [ ]:
clf.fit(X_train,Y_train)

In [ ]:
clf.best_params_

In [ ]:
Y_pred=clf.predict(X_test)
Y_pred_train=clf.predict(X_train)

In [ ]:
clf.score(X_train,Y_train)

In [ ]:
clf.score(X_test,Y_test)

In [ ]:
r2=r2_score(Y_test,Y_pred)
r2

# DECISION TREE MODEL


In [ ]:
X=df.drop(columns="Predicted Wins")
Y=df["Predicted Wins"]

In [ ]:
from sklearn.tree import DecisionTreeRegressor

In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.25,random_state=21)

In [ ]:
dt=DecisionTreeRegressor()
dt.fit(X_train,Y_train)

In [ ]:
Y_pred=dt.predict(X_test)

In [ ]:
r2=r2_score(Y_test,Y_pred)
r2

In [ ]:
plt.scatter(Y_test,Y_pred)
plt.xlabel('actual data')
plt.ylabel('predicted data')
plt.show()

# DECISION TREE WITH HYPER PARAMETER TUNING

In [ ]:
X=df.drop(columns="Predicted Wins")
Y=df["Predicted Wins"]

In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.25,random_state=21)

In [ ]:
dt=DecisionTreeRegressor()


In [ ]:
param_grid = {
               'max_depth': range(3,12,2),
               'min_samples_split': range(4,10,2),
               'min_samples_leaf': range(3,10,2),
              'criterion':["mse", "friedman_mse", "mae", "poisson"]}

In [ ]:
clf=GridSearchCV(dt,param_grid,cv=5,n_jobs=-1)

In [ ]:
clf.fit(X_train,Y_train)

In [ ]:
clf.best_params_

In [ ]:
Y_pred=clf.predict(X_test)


In [ ]:
clf.score(X_train,Y_train)

In [ ]:
clf.score(X_test,Y_test)

In [ ]:
r2=r2_score(Y_test,Y_pred)
r2

In [ ]:
plt.scatter(Y_test,Y_pred)
plt.xlabel('actual data')
plt.ylabel('predicted data')
plt.show()

# Modeling Using Gradient Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
X=df.drop(columns="Predicted Wins")
Y=df["Predicted Wins"]

In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.25,random_state=21)

In [ ]:
gbdt=GradientBoostingRegressor()

In [ ]:
gbdt.fit(X_train,Y_train)

In [ ]:
Y_pred=gbdt.predict(X_test)


In [ ]:
r2=r2_score(Y_test,Y_pred)
r2

In [ ]:
plt.scatter(Y_test,Y_pred)
plt.xlabel('actual data')
plt.ylabel('predicted data')
plt.show()

In [ ]:
md=[lm,knn,dt,gbdt]
import pickle
filename="Baseball.pkl"
pickle.dump(md,open(filename,"wb"))

# Conclusion:
Conducted 4 models on BaseBall dataset namely, Linear Regression, KNN, Desicion Tree and Gradient Boosting with Hyperparameter tuning
It comes to a conclusion that Linear Regression model is providing best score for BaseBall dataset
r2_score=0.0.8245656778621684